## In this chapter, you will become acquainted with the fundamentals of Relational Databases and the Relational Model. You will learn how to connect to a database and then interact with it by writing basic SQL queries, both in raw SQL as well as with SQLAlchemy, which provides a Pythonic way of interacting with databases

## Engines and Connection Strings
Alright, it's time to create your first engine! An engine is just a common interface to a database, and the information it requires to connect to one is contained in a connection string, such as 
`sqlite:///census_nyc.sqlite`. Here, `sqlite` is the database driver, while `census_nyc.sqlite` is a SQLite file contained in the local directory.

You can learn a lot more about connection strings in the SQLAlchemy documentation.


Your job in this exercise is to create an engine that connects to a local SQLite file named census.sqlite. Then, print the names of the tables it contains using the .table_names() method. Note that when you just want to print the table names, you do not need to use engine.connect() after creating the engine.

In [2]:
# ! pip install SQLAlchemy

In [3]:
# Import create_engine
from sqlalchemy import create_engine 

# Create an engine that connects to the census.sqlite file: engine
engine = create_engine('sqlite:///census.sqlite')

# Print table names
engine.table_names()

['census', 'state_fact']

## Autoloading Tables from a Database
SQLAlchemy can be used to automatically load tables from a database using something called reflection. Reflection is the process of reading the database and building the metadata based on that information. It's the opposite of creating a Table by hand and is very useful for working with existing databases. To perform reflection, you need to import the Table object from the SQLAlchemy package. Then, you use this Table object to read your table from the engine and autoload the columns. Using the Table object in this manner is a lot like passing arguments to a function. For example, to autoload the columns with the engine, you have to specify the keyword arguments autoload=True and autoload_with=engine to Table().

In this exercise, your job is to reflect the census table available on your engine into a variable called census. The metadata has already been loaded for you using MetaData() and is available in the variable metadata

In [4]:
from sqlalchemy import Table, MetaData

In [5]:
metadata = MetaData(bind = None)

In [6]:
# Reflect census table from the engine: census
census = Table('census', metadata, autoload=True, autoload_with=engine)

# Print census table metadata
repr(census)

"Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)"

## Viewing Table Details
Great job reflecting the census table! Now you can begin to learn more about the columns and structure of your table. It is important to get an understanding of your database by examining the column names. This can be done by using the .columns attribute and accessing the .keys() method. For example, census.columns.keys() would return a list of column names of the census table.

Following this, we can use the metadata container to find out more details about the reflected table such as the columns and their types. For example, table objects are stored in the metadata.tables dictionary, so you can get the metadata of your census table with metadata.tables['census']. This is similar to your use of the repr() function on the census table from the previous exercise.

In [7]:
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [8]:
# Reflect the census table from the engine: census
census = Table('census', metadata, autoload=True, autoload_with=engine)

# Print the column names
census.columns.keys()

# Print full table metadata
repr(metadata.tables['census'])

['state', 'sex', 'age', 'pop2000', 'pop2008']

"Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)"

## Selecting data from a Table: raw SQL
Using what we just learned about SQL and applying the .execute() method on our connection, we can leverage a raw SQL query to query all the records in our census table. The object returned by the .execute() method is a ResultProxy. On this ResultProxy, we can then use the .fetchall() method to get our results - that is, the ResultSet.

In this exercise, you'll use a traditional SQL query. In the next exercise, you'll move to SQLAlchemy and begin to understand its advantages. Go for it!

In [9]:
connection = engine.connect()

In [10]:
# Build select statement for census table: stmt
stmt = 'SELECT * from census'

# Execute the statement and fetch the results: results
results = connection.execute(stmt).fetchall()

# Print results
results

[('Illinois', 'M', 0, 89600, 95012),
 ('Illinois', 'M', 1, 88445, 91829),
 ('Illinois', 'M', 2, 88729, 89547),
 ('Illinois', 'M', 3, 88868, 90037),
 ('Illinois', 'M', 4, 91947, 91111),
 ('Illinois', 'M', 5, 93894, 89802),
 ('Illinois', 'M', 6, 93676, 88931),
 ('Illinois', 'M', 7, 94818, 90940),
 ('Illinois', 'M', 8, 95035, 86943),
 ('Illinois', 'M', 9, 96436, 86055),
 ('Illinois', 'M', 10, 97280, 86565),
 ('Illinois', 'M', 11, 94029, 86606),
 ('Illinois', 'M', 12, 92402, 89596),
 ('Illinois', 'M', 13, 89926, 91661),
 ('Illinois', 'M', 14, 90717, 91256),
 ('Illinois', 'M', 15, 92178, 92729),
 ('Illinois', 'M', 16, 90587, 93083),
 ('Illinois', 'M', 17, 92782, 94541),
 ('Illinois', 'M', 18, 90997, 100253),
 ('Illinois', 'M', 19, 89629, 96588),
 ('Illinois', 'M', 20, 91040, 95460),
 ('Illinois', 'M', 21, 85176, 91373),
 ('Illinois', 'M', 22, 84372, 90723),
 ('Illinois', 'M', 23, 85529, 91982),
 ('Illinois', 'M', 24, 84549, 90237),
 ('Illinois', 'M', 25, 87630, 95217),
 ('Illinois', 'M', 26

## Selecting data from a Table with SQLAlchemy
Excellent work so far! It's now time to build your first select statement using SQLAlchemy. SQLAlchemy provides a nice "Pythonic" way of interacting with databases. So rather than dealing with the differences between specific dialects of traditional SQL such as MySQL or PostgreSQL, you can leverage the Pythonic framework of SQLAlchemy to streamline your workflow and more efficiently query your data. For this reason, it is worth learning even if you may already be familiar with traditional SQL.

In this exercise, you'll once again build a statement to query all records from the census table. This time, however, you'll make use of the select() function of the sqlalchemy module. This function requires a list of tables or columns as the only required argument.

Table and MetaData have already been imported. The metadata is available as metadata and the connection to the database as connection.

__Instructions__
- Import select from the sqlalchemy module.
- Reflect the census table. This code is already written for you.
- Create a query using the select() function to retrieve the census table. To do so, pass a list to   select() containing a single element: census.
- Print stmt to see the actual SQL query being created. This code has been written for you.
- Using the provided print() function, print all the records from the census table. To do this:
  - Use the .execute() method on connection with stmt as the argument to retrieve the ResultProxy.
  - Use .fetchall() on connection.execute(stmt) to retrieve the ResultSet.

In [11]:
# Import select
from sqlalchemy import select

# Reflect census table via engine: census
census = Table('census', metadata, autoload=True, autoload_with=engine)

# Build select statement for census table: stmt
stmt = select([census])

# Print the emitted statement to see the SQL emitted
stmt

# Execute the statement and print the results
connection.execute(stmt).fetchall()

<sqlalchemy.sql.selectable.Select at 0x6db2cf0; Select object>

[('Illinois', 'M', 0, 89600, 95012),
 ('Illinois', 'M', 1, 88445, 91829),
 ('Illinois', 'M', 2, 88729, 89547),
 ('Illinois', 'M', 3, 88868, 90037),
 ('Illinois', 'M', 4, 91947, 91111),
 ('Illinois', 'M', 5, 93894, 89802),
 ('Illinois', 'M', 6, 93676, 88931),
 ('Illinois', 'M', 7, 94818, 90940),
 ('Illinois', 'M', 8, 95035, 86943),
 ('Illinois', 'M', 9, 96436, 86055),
 ('Illinois', 'M', 10, 97280, 86565),
 ('Illinois', 'M', 11, 94029, 86606),
 ('Illinois', 'M', 12, 92402, 89596),
 ('Illinois', 'M', 13, 89926, 91661),
 ('Illinois', 'M', 14, 90717, 91256),
 ('Illinois', 'M', 15, 92178, 92729),
 ('Illinois', 'M', 16, 90587, 93083),
 ('Illinois', 'M', 17, 92782, 94541),
 ('Illinois', 'M', 18, 90997, 100253),
 ('Illinois', 'M', 19, 89629, 96588),
 ('Illinois', 'M', 20, 91040, 95460),
 ('Illinois', 'M', 21, 85176, 91373),
 ('Illinois', 'M', 22, 84372, 90723),
 ('Illinois', 'M', 23, 85529, 91982),
 ('Illinois', 'M', 24, 84549, 90237),
 ('Illinois', 'M', 25, 87630, 95217),
 ('Illinois', 'M', 26

## Handling a ResultSet
Recall the differences between a ResultProxy and a ResultSet:

ResultProxy: The object returned by the .execute() method. It can be used in a variety of ways to get the data returned by the query.
ResultSet: The actual data asked for in the query when using a fetch method such as .fetchall() on a ResultProxy.
This separation between the ResultSet and ResultProxy allows us to fetch as much or as little data as we desire.

Once we have a ResultSet, we can use Python to access all the data within it by column name and by list style indexes. For example, you can get the first row of the results by using results[0]. With that first row then assigned to a variable first_row, you can get data from the first column by either using first_row[0] or by column name such as first_row['column_name']. You'll now practice exactly this using the ResultSet you obtained from the census table in the previous exercise. It is stored in the variable results. Enjoy!



In [12]:
# Get the first row of the results by using an index: first_row
first_row = results[0]

# Print the first row of the results
first_row
      
# Print the first column of the row by using an index
first_row[0]

# Print the state column of the row by using its name
first_row['state']

('Illinois', 'M', 0, 89600, 95012)

'Illinois'

'Illinois'

In [13]:
census

Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)

In [15]:
engine

Engine(sqlite:///census.sqlite)